In [13]:
import os
import glob
from PIL import Image
from tqdm import tqdm

In [14]:
def load_yolo_label(label_path, image_shape):
    answer_list = []
    # width, height = image.size
    image_width, image_height = image_shape
    with open(label_path, 'r') as f:
        for line in f.readlines():
            dict_ = {}
            label, x_center, y_center, width, height, score = line.split(' ')
            x_center, y_center, width, height = float(x_center), float(y_center), float(width), float(height)
            width, height = float(width*image_width), float(height*image_height)
            x_center, y_center,  = float(x_center*image_width), float(y_center*image_height)
            x1, y1 = float(x_center-width/2), float(y_center-height/2)
            # [x_center/image_width, y_center/image_height, width/image_width, height/image_height]
            image_name = os.path.basename(label_path)
            dict_['image_id'] = int(image_name.split('.')[0])
            dict_['bbox'] = [x1, y1, width, height]
            dict_['score'] = float(score)
            dict_['category_id'] = int(label) if int(label) < 10 else 0

            answer_list.append(dict_)
    return answer_list


In [15]:
exp = 'exp18'

label_dir = os.path.join('runs', 'detect', exp, 'labels', '*.txt')
label_path_list = glob.glob(label_dir)
label_path_list = sorted(label_path_list)
# image_name_list = list(map(os.path.basename, label_path_list))

image_dir = os.path.join('..', 'data', 'test')
# image_path_list = glob.glob(image_dir)
# image_path_list = sorted(image_path_list)
# print(list(map(os.path.basename, image_path_list))[:5])

answer = []

for label_path in tqdm(label_path_list):
    image_name = os.path.basename(label_path).replace('.txt', '.png')
    image_path = os.path.join(image_dir, image_name)
    image = Image.open(image_path)
    answer += load_yolo_label(label_path, image.size)

answer = sorted(answer, key=lambda x: x['image_id'])

100%|██████████| 12953/12953 [00:37<00:00, 345.36it/s]


In [16]:
import json
dir_path = f'answer-yolov5-{exp}'
os.mkdir(dir_path)
print(json.dumps(answer, indent=4), file=open(os.path.join(dir_path, f'answer.json'), 'w'))